# AIM Hackathon: Sample code
19.10.2024

In [28]:
import os
import requests
import PyPDF2
import tiktoken
import pandas as pd
import pickle
from dotenv import load_dotenv
from tqdm import tqdm

from langchain_openai import ChatOpenAI
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai.embeddings.base import OpenAIEmbeddings

from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS

from Cryptodome.Cipher import AES


# load openai key
if not load_dotenv():
    raise Exception('Error loading .env file. Make sure to place a valid OPEN_AI_KEY in the .env file.')

In [18]:
REPORTS_SAVE_PATH = 'data/sample_reports'
DB_PATH = "data/db/sample.db"

# See https://openai.com/api/pricing/
MODEL = "gpt-4o"

In [19]:
df = pd.read_json('data/reports.json')
df

,company_name,year,dataset,pdf_url
0,Walmart,2023,handcrafted,https://corporate.walmart.com/content/dam/corp...
1,Walmart,2021,handcrafted,https://corporate.walmart.com/content/dam/corp...
2,Walmart,2019,handcrafted,https://corporate.walmart.com/content/dam/corp...
3,Amazon,2023,handcrafted,https://sustainability.aboutamazon.com/content...
4,Amazon,2021,handcrafted,https://sustainability.aboutamazon.com/content...
...,...,...,...,...
141,tarkett,2020,scraped,https://www.tarkett.com/sites/default/files/20...
142,trivium-packaging,2021,scraped,https://www.triviumpackaging.com/media/13fl4q3...
143,trivium-packaging,2020,scraped,https://triviumpackaging.com/sustainability/re...
144,trust,2023,scraped,https://dezlwerqy1h00.cloudfront.net/images/co...


## Download some reports

In [20]:
# EXAMPLE: select apple reports
df_sample = df[df['dataset']=='handcrafted']

In [53]:
# Storing the encryption keys for further decryption
enc_keys = []

# download Apple reports to save_dir
def download_files(df: pd.DataFrame, save_dir: str):
    os.makedirs(save_dir, exist_ok=True)
    for url in df['pdf_url']:
        pdf_filename = os.path.basename(url)
        # Checking if the file is encrypted
        if('?' in pdf_filename):
            # Saving the password for decryption
            enc_keys.append(pdf_filename)
            # Removing question mark
            pdf_filename = pdf_filename.split('?')[0]
            
        response = requests.get(url)
        with open(os.path.join(save_dir, pdf_filename), 'wb') as file:
            file.write(response.content)
    print(f"Success.")

In [54]:
download_files(df_sample, REPORTS_SAVE_PATH)

Success.


In [46]:
# Checking the encryption keys
enc_keys

['2023_Volkswagen_Group_Sustainability_Report.pdf?1710947082',
 'Nonfinancial_Report_2021_en.pdf?1682331247',
 'Nonfinancial_Report_2019_en.pdf?1682331576']

In [27]:
pip install pycryptodomex

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Create simple vector database

In [55]:
def get_password(f):
    for tmp in enc_keys:
        if(f == tmp.split()[0]):
            return tmp

def get_documents_from_path(files_path: str) -> [Document]:
    documents = []
    
    for file in os.listdir(files_path):
        _, file_extension = os.path.splitext(file)
        text = ""
        
        if file_extension == ".pdf":
            with open(os.path.join(files_path, file), 'rb') as f:
                reader = PyPDF2.PdfReader(f, strict=False)
                
                if reader.is_encrypted:
                    try:
                        # Try to decrypt with the provided password (or an empty string if no password is given)
                        pdf_password = get_password(file)
                        
                        if pdf_password:
                            success = reader.decrypt(pdf_password)
                        else:
                            success = reader.decrypt("")

                        if success == 0:
                            print(f"Failed to decrypt {file}: Invalid password.")
                            continue  # Skip file if decryption fails
                        else:
                            print(f"Decrypted {file} successfully.")
                    except Exception as e:
                        print(f"Failed to decrypt {file}: {e}")
                        continue  # Skip file if decryption fails
                for page in reader.pages:
                    text += page.extract_text() + "\n"
                
            if text:
                documents.append(Document(page_content=text, metadata={"source": file}))
            else:
                print(f"WARNING: No text extracted from {file}")
        else:
            # TODO: can add support for other file types here
            raise Exception(f"Unsupported file extension: {file_extension}")
    
    return documents

In [56]:
documents = get_documents_from_path(REPORTS_SAVE_PATH)

Decrypted 2023_Volkswagen_Group_Sustainability_Report.pdf successfully.
Decrypted bp-sustainability-report-2021.pdf successfully.
Decrypted bp-sustainability-report-2023.pdf successfully.
Decrypted Nonfinancial_Report_2021_en.pdf successfully.


In [57]:
# TODO could also just provide a dummy retriever to not spoil too much
class DummyRetriever:
    def __init__(self, texts):
        self.texts = texts
        
    def dummy_retriever(self, query):
        import random
        return random.sample(self.texts, k=3)

In [58]:
# Create database
text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=300, separators=["\n\n", "\n"])

# split documents and create vector database
texts = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings()  # https://platform.openai.com/docs/guides/embeddings/embedding-models
db = FAISS.from_documents(texts, embeddings)

# count build embedding token number
tokenizer = tiktoken.get_encoding("cl100k_base")
build_token_count = sum([len(tokenizer.encode(doc.page_content)) for doc in texts])
print(f"Token count: {build_token_count}")

Token count: 1286489


In [60]:
# Store the database
with open(DB_PATH, "wb") as f:
    pickle.dump(db.serialize_to_bytes(), f)

## Create simple RAG

In [20]:
# Load the database
DB_PATH = "data/db/sample.db"

with open(DB_PATH, "rb") as f:
    db_bytes = pickle.load(f)
    db = FAISS.deserialize_from_bytes(db_bytes, OpenAIEmbeddings(), allow_dangerous_deserialization=True)

In [ ]:
# Load the LLM
llm = ChatOpenAI(model_name=MODEL, temperature=0)  # for deterministic outputs

system_prompt = """
You are an expert assistant. Use only the following retrieved context to answer the question accurately and concisely.
Provide your answer as a **number only**, without any additional text or explanation.
If nothing is mentioned in the context, say "I don't know".
Context: {context}
Question: {question}
"""


prompt_template = PromptTemplate(
    input_variables=["context", "question"], 
    template=system_prompt
)

retrieval_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=db.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt_template}
)

In [51]:
def ask_question(query):
    response = retrieval_chain({"query": query})
    print(f"Question: {query}\nAnswer: {response['result']}")
    return response

In [59]:
response = ask_question("How many companies gave a report in 2021?")

Question: How many companies gave a report in 2021?
Answer: I don't know.
